In [24]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *

In [25]:
### get file names ###
env = 'local'
files = os.listdir('./database/')
filenames = []

for i in range(len(files)):
    if 'no' in files[i]:
        if '__' not in files[i]:
            if 'all' not in files[i]:
                filenames.append(files[i])
                print(i, files[i])

17 Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj
21 Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj
25 Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj
31 Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj
35 Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj
39 Cu_100_u2_RPBE-D2_no020_CO_n1_d8.traj
45 Cu_100_u2_RPBE_no001_CO_n1_d8.traj
49 Cu_100_u2_RPBE_no010_CO_n1_d8.traj
53 Cu_100_u2_RPBE_no020_CO_n1_d8.traj
59 Cu_100_u2_optB88-vdW_no001_CO_n1_d8.traj
63 Cu_100_u2_optB88-vdW_no010_CO_n1_d8.traj
67 Cu_100_u2_optB88-vdW_no020_CO_n1_d8.traj
74 Cu_100_u2_vdW-DF2_no001_CO_n1_d8.traj
78 Cu_100_u2_vdW-DF2_no010_CO_n1_d8.traj
82 Cu_100_u2_vdW-DF2_no020_CO_n1_d8.traj
87 Cu_100_u2_vdW-DF_no001_CO_n1_d8.traj
91 Cu_100_u2_vdW-DF_no010_CO_n1_d8.traj
95 Cu_100_u2_vdW-DF_no020_CO_n1_d8.traj
101 Cu_111_u2_BEEF-vdW_no001_CO_n1_d8.traj
105 Cu_111_u2_BEEF-vdW_no008_CO_n1_d8.traj
109 Cu_111_u2_BEEF-vdW_no015_CO_n1_d8.traj
113 Cu_111_u2_BEEF-vdW_no020_CO_n1_d8.traj
119 Cu_111_u2_RPBE-D2_no001_CO_n1_d8.traj
123 Cu_111_u2_RPBE-D2_no008_CO_n1_d8.traj
127 Cu

In [18]:
for file in filenames:
    try:
        A = make_database(file)
        A.delete_from_database()
    except:
        print(file, 'Some error occured.')

Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj deleted
Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj deleted
Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj deleted
Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj deleted
Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj deleted
Cu_100_u2_RPBE-D2_no020_CO_n1_d8.traj deleted
Cu_100_u2_RPBE_no001_CO_n1_d8.traj deleted
Cu_100_u2_RPBE_no010_CO_n1_d8.traj deleted
Cu_100_u2_RPBE_no020_CO_n1_d8.traj deleted
Cu_100_u2_optB88-vdW_no001_CO_n1_d8.traj deleted
Cu_100_u2_optB88-vdW_no010_CO_n1_d8.traj deleted
Cu_100_u2_optB88-vdW_no020_CO_n1_d8.traj deleted
Cu_100_u2_vdW-DF2_no001_CO_n1_d8.traj deleted
Cu_100_u2_vdW-DF2_no010_CO_n1_d8.traj deleted
Cu_100_u2_vdW-DF2_no020_CO_n1_d8.traj deleted
Cu_100_u2_vdW-DF_no001_CO_n1_d8.traj deleted
Cu_100_u2_vdW-DF_no010_CO_n1_d8.traj deleted
Cu_100_u2_vdW-DF_no020_CO_n1_d8.traj deleted
Cu_111_u2_BEEF-vdW_no001_CO_n1_d8.traj deleted
Cu_111_u2_BEEF-vdW_no008_CO_n1_d8.traj deleted
Cu_111_u2_BEEF-vdW_no015_CO_n1_d8.traj deleted
Cu_111_u2_BEEF-vdW_no020_CO_n1_

In [19]:
for file in filenames:
    try:
        A = make_database(file)
        A.add_to_database()
        A.update_database_Eeach()
        A.check_database()
    except:
        print(file, 'Some error occured.')

Not in database.
Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj
One post: 5e5acdf77d4eccacdaef0451

Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj Some error occured.
Not in database.
Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj
One post: 5e5acdf87d4eccacdaef0453

Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj Some error occured.
Not in database.
Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj
One post: 5e5acdf87d4eccacdaef0455

Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj Some error occured.
Not in database.
Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj
One post: 5e5acdf97d4eccacdaef0457

Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj Some error occured.
Not in database.
Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj
One post: 5e5acdf97d4eccacdaef0459

Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj Some error occured.
Not in database.
Cu_100_u2_RPBE-D2_no020_CO_n1_d8.traj
One post: 5e5acdf97d4eccacdaef045b

Cu_100_u2_RPBE-D2_no020_CO_n1_d8.traj Some error occured.
Not in database.
Cu_100_u2_RPBE_no001_CO_n1_d8.traj
One post: 5e5acdfa7d4eccacdaef045d

Cu_100_u2_RP

Pt_100_u2_vdW-DF2_no010_CO_n1_d9.traj
One post: 5e5ace107d4eccacdaef04bf

Pt_100_u2_vdW-DF2_no010_CO_n1_d9.traj Some error occured.
Not in database.
Pt_100_u2_vdW-DF2_no020_CO_n1_d9.traj
One post: 5e5ace107d4eccacdaef04c1

Pt_100_u2_vdW-DF2_no020_CO_n1_d9.traj Some error occured.
Not in database.
Pt_100_u2_vdW-DF_no001_CO_n1_d9.traj
One post: 5e5ace117d4eccacdaef04c3

Pt_100_u2_vdW-DF_no001_CO_n1_d9.traj Some error occured.
Not in database.
Pt_100_u2_vdW-DF_no010_CO_n1_d9.traj
One post: 5e5ace117d4eccacdaef04c5

Pt_100_u2_vdW-DF_no010_CO_n1_d9.traj Some error occured.
Not in database.
Pt_100_u2_vdW-DF_no020_CO_n1_d9.traj
One post: 5e5ace117d4eccacdaef04c7

Pt_100_u2_vdW-DF_no020_CO_n1_d9.traj Some error occured.
Not in database.
Pt_111_u2_BEEF-vdW_no001_CO_n1_d9.traj
One post: 5e5ace127d4eccacdaef04c9

Pt_111_u2_BEEF-vdW_no001_CO_n1_d9.traj Some error occured.
Not in database.
Pt_111_u2_BEEF-vdW_no008_CO_n1_d9.traj
One post: 5e5ace137d4eccacdaef04cb

Pt_111_u2_BEEF-vdW_no008_CO_n1_d9.t

In [21]:
for file in filenames:
    try:
        A = make_database(file)
        data = A.check_database()
        print(data['name'], data['totaladsE'])
    except:
        print(file, 'Some error occured.')

Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj 41.776402989999994
Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj 41.99910281999999
Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj 41.85465076
Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj -0.532286820000003
Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj -5.263199929999999
Cu_100_u2_RPBE-D2_no020_CO_n1_d8.traj -0.4179188399999987
Cu_100_u2_RPBE_no001_CO_n1_d8.traj -0.5497481399999913
Cu_100_u2_RPBE_no010_CO_n1_d8.traj -0.48469112999999453
Cu_100_u2_RPBE_no020_CO_n1_d8.traj -0.397524269999991
Cu_100_u2_optB88-vdW_no001_CO_n1_d8.traj -3.209185550000001
Cu_100_u2_optB88-vdW_no010_CO_n1_d8.traj -3.1580763099999984
Cu_100_u2_optB88-vdW_no020_CO_n1_d8.traj -3.0864949999999993
Cu_100_u2_vdW-DF2_no001_CO_n1_d8.traj -3.304798069999997
Cu_100_u2_vdW-DF2_no010_CO_n1_d8.traj -3.2134275999999957
Cu_100_u2_vdW-DF2_no020_CO_n1_d8.traj -3.122851659999995
Cu_100_u2_vdW-DF_no001_CO_n1_d8.traj -3.6569209099999966
Cu_100_u2_vdW-DF_no010_CO_n1_d8.traj -3.5840128999999976
Cu_100_u2_vdW-DF_no020_CO_n1_

In [7]:
for file in filenames:
    try:
        A = make_database(file)
        A.update_database_adsorbates_correlation()
    except:
        print(file, 'Some error occured.')

Pd_111_u1_RPBE_no001_CO_n1_d3.traj Some error occured.
Pd_111_u1_RPBE_no002_CO_n1_d3.traj Some error occured.
Pd_111_u1_RPBE_no003_CO_n1_d3.traj Some error occured.
Pd_111_u1_RPBE_no004_CO_n1_d3.traj Some error occured.
Pd_111_u2_RPBE_no001_CO_n1_d9.traj Some error occured.
Pd_111_u2_RPBE_no002_CO_n2_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no003_CO_n3_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no004_CO_n4_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no005_CO_n2_d4.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no006_CO_n3_d4.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no007_CO_n2_d4.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no008_CO_n1_d9.traj Some error occured.
Pd_111_u2_RPBE_no009_CO_n2_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no010_CO_n3_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no011_CO_n4_d3.traj Adsorbate correlation updated.
Pd_111_u2_RPBE_no012_CO_n2_d3.traj Adsorbate correlation updated.
Pd_111_u2_

In [22]:
# for file in filenames:
#     try:
#         A = make_database(file)
#         print(A.check_database()['minimum_distance'])
#     except:
#         print(file, 'Some error occured.')

In [23]:
file = 'Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj'
A = make_database(file)
A.check_database()

{'_id': ObjectId('5e5acdf87d4eccacdaef0455'),
 'name': 'Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj',
 'isvalid': 'yes',
 'element': 'Cu',
 'face': '100',
 'unitlength': 2,
 'xc': 'BEEF-vdW',
 'adsorbate': 'CO',
 'numberofads': 1,
 'coverage': 0.25,
 'surfatomnum': 4,
 'E': -18.28520232,
 'bareE': -48.0018866,
 'E_CO': -12.13796648,
 'totaladsE': 41.85465076,
 'aveadsE/suratom': 10.46366269,
 'aveadsE/ads': 41.85465076,
 'E_int_space': -0.0045082500000006576,
 'E_each_ads': None,
 'area': 26.82487613815005,
 'density': 0.03727883009971519,
 'igroups': [2],
 'rgroups': [2],
 'COlengthlis': [1.1686617418332206],
 'converged': 'yes',
 'not_moved': 'yes',
 'kept_sites': 'yes',
 'minimum_distance': None,
 'ads_dist2': None,
 'ads_dist3': None}

In [15]:
file = 'Cu_100_u2_RPBE_no020_CO_n1_d8.traj'
A = make_database(file)
A.check_database()

{'_id': ObjectId('5e59f4dc7d4eccacdaef02d1'),
 'name': 'Cu_100_u2_RPBE_no020_CO_n1_d8.traj',
 'isvalid': 'yes',
 'element': 'Cu',
 'face': '100',
 'unitlength': 2,
 'xc': 'RPBE',
 'adsorbate': 'CO',
 'numberofads': 1,
 'coverage': 0.25,
 'surfatomnum': 4,
 'E': -62.8509433,
 'bareE': -47.96798272,
 'E_CO': -14.48543631,
 'totaladsE': -0.397524269999991,
 'aveadsE/suratom': -0.09938106749999776,
 'aveadsE/ads': -0.397524269999991,
 'E_int_space': 0.0014228000000002794,
 'E_each_ads': None,
 'area': 27.044159151757036,
 'density': 0.03697656097897319,
 'igroups': [2],
 'rgroups': [2],
 'COlengthlis': [1.203542351539886],
 'converged': 'yes',
 'not_moved': 'yes',
 'kept_sites': 'yes',
 'minimum_distance': None,
 'ads_dist2': None,
 'ads_dist3': None}

In [27]:
A = make_database('Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj')
A.check_database()

{'_id': ObjectId('5e5acdf77d4eccacdaef0451'),
 'name': 'Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj',
 'isvalid': 'no',
 'element': 'Cu',
 'face': '100',
 'unitlength': 2,
 'xc': 'BEEF-vdW',
 'adsorbate': 'CO',
 'numberofads': 1,
 'coverage': 0.25,
 'surfatomnum': 4,
 'E': -18.36345009,
 'bareE': -48.0018866,
 'E_CO': -12.13796648,
 'totaladsE': 41.776402989999994,
 'aveadsE/suratom': 10.444100747499999,
 'aveadsE/ads': 41.776402989999994,
 'E_int_space': -0.00450823999999983,
 'E_each_ads': None,
 'area': 26.82487613815005,
 'density': 0.03727883009971519,
 'igroups': [0],
 'rgroups': [0],
 'COlengthlis': [1.1546559422838865],
 'converged': 'no',
 'not_moved': 'no',
 'kept_sites': 'yes',
 'minimum_distance': None,
 'ads_dist2': None,
 'ads_dist3': None}

In [30]:
atoms = query('Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj','local')

In [31]:
atoms.get_potential_energy()

-18.36345009